## KITTI

## Install modules

In [43]:
# %cd /home/kyungpyo/git/voxelnet
# %pip install -e .

## Import modules

In [44]:
%load_ext autoreload
%autoreload 2

import yaml
from visualization.plotly_wrapper import *
import pathlib
from data.kitti.dataset import KittiDataset
import utils.utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load dataset and visualization

### Point cloud and label

In [54]:
config_path = pathlib.Path('/home/kyungpyo/git/voxelnet/config/voxelnet_kitti.yaml')

cfg = None
with open(config_path) as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

dataset1 = KittiDataset('/home/kyungpyo/git/voxelnet/data/training', cfg)
data = dataset1[2]

lidar = data['lidar']
label = data['label']
lidar_dict = {
    'x': lidar[:, 0],
    'y': lidar[:, 1],
    'z': lidar[:, 2],
    'i': lidar[:, 3],
}
pointcloud = goScatterLidar(lidar_dict)
data = [pointcloud]

for key in label.keys():
    if key == 'Car':
        for box in label[key]:
            data.append(goCube(box))

layout = goLayout('lidar point cloud with label')
goFigure(data, layout)

KittiDataset:: voxel number: W: 352
KittiDataset:: voxel number: H: 400
KittiDataset:: voxel number: D: 10
KittiDataset:: anchor size: (200, 176, 2, 7)


### Target encoding and voxelization verification

In [55]:
data = dataset1[2]
target = data['target']
pos_equal_one = data['pos_equal_one']
neg_equal_one = data['neg_equal_one']
anchor = dataset1.anchors
_target = target[pos_equal_one == 1]
_anchor = anchor[pos_equal_one == 1]
_target = utils.anchorDecoder(_anchor, _target)


In [56]:
lidar = data['lidar']
label = data['label']
lidar_dict = {
    'x': lidar[:, 0],
    'y': lidar[:, 1],
    'z': lidar[:, 2],
    'i': lidar[:, 3],
}
pointcloud = goScatterLidar(lidar_dict)
_data = [pointcloud]

for t in _target:
    box = {}
    box['x'] = t[0]
    box['y'] = t[1]
    box['z'] = t[2]
    box['w'] = t[3]
    box['l'] = t[4]
    box['h'] = t[5]
    box['yaw'] = t[6]
    _data.append(goCube(box))

layout = goLayout('lidar point cloud with label')
goFigure(_data, layout)

In [49]:
voxel_coords = data['voxel_coords']
voxel_coords = voxel_coords.astype(np.float32)
voxel_coords = voxel_coords * (dataset1.voxel_size_x, 
                               dataset1.voxel_size_y, 
                               dataset1.voxel_size_z) \
                            + (dataset1.range_x[0], 
                               dataset1.range_y[0], 
                               dataset1.range_z[0])
voxel_coord_dict = {
    'x': voxel_coords[:,0],
    'y': voxel_coords[:,1],
    'z': voxel_coords[:,2],
    'i': voxel_coords[:,2],
}
pointcloud = goScatterLidar(voxel_coord_dict)
_data = [pointcloud]
layout = goLayout('lidar point cloud with label')
goFigure(_data, layout)